### Performance reports

In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
# preparation database
import sqlite3

txhash_path = 'txhash.db'
conn = sqlite3.connect(txhash_path)
cur = conn.cursor()
index_path = './completed/index.db'
iconn = sqlite3.connect(index_path)
icur = iconn.cursor()

def create_meta_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Meta (
                     key TEXT PRIMARY KEY,
                     value TEXT);
                ''')

def create_txin_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxIn (
                     tx INTEGER,
                     n INTEGER,
                     addr INTEGER);
                ''')

def create_txout_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxOut (
                     tx INTEGER,
                     n INTEGER,
                     addr INTEGER);
                ''')

def insert_txin(tx, n, addr):
    cur.execute('''INSERT OR IGNORE INTO TxIn (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                ''', (tx, n, addr))

def insert_txins(iaddr):
    cur.executemany('''INSERT OR IGNORE INTO TxIn (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                    ''', iaddr)
    
def insert_txout(tx, n, addr):
    cur.execute('''INSERT OR IGNORE INTO TxOut (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                ''', (tx, n, addr))

    
def insert_txouts(oaddr):
    cur.executemany('''INSERT OR IGNORE INTO TxOut (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                    ''', oaddr)
    
def txn2addrid(tx, n):
    cur.execute('''SELECT addr FROM TxOut WHERE
                       tx = ? AND n = ?
                ''', (tx, n))
    return cur.fetchone()[0]

def update_meta(key, value):
    cur.execute('''INSERT OR IGNORE INTO Meta (
                        key, value) VALUES (
                        ?, ?);
                ''', (key, value))
    cur.execute('''UPDATE Meta SET value = ? WHERE key = ?;
                ''', (value, key))

def get_blkid(blkhash):
    icur.execute('''SELECT id FROM BlkID WHERE blkhash = ?''', (blkhash,))
    return icur.fetchone()[0]
    
def get_txid(txhash):
    icur.execute('''SELECT id FROM TxID WHERE txhash = ?''', (txhash,))
    return icur.fetchone()[0]
    
def get_addrid(addr):
    icur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (addr,))
    return icur.fetchone()[0]
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()
    
def get_max_height():
    icur.execute('''SELECT MAX(id) FROM BlkID;''')
    return icur.fetchone()[0] 
    
def get_meta(key):
    cur.execute('''SELECT value FROM Meta WHERE key = ?''', (key,))
    result = cur.fetchone()
    if result is not None:
        result = result[0]
    return result

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
# timeout = 60*5

def get_rpc():
#     return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}')

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
create_meta_table()
create_txin_table()
create_txout_table()

In [ ]:
def get_addresses_in(theight):
    rpc_connection = get_rpc()
    iaddresses = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    blkid = get_blkid(block_hash)
    for tx in block['tx']:
        txid = get_txid(tx['hash'])
        n = -1
        for vin in tx['vin']:
            n = n + 1
            if 'coinbase' in vin.keys():
                iaddresses.append((txid, n, 0))
            else:
                ptxid = get_txid(vin['txid'])
                addrid = txn2addrid(ptxid, vin['vout'])
                iaddresses.append((txid, n, addrid))
    return iaddresses

def get_addresses_out(theight):
    rpc_connection = get_rpc()
    oaddresses = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    blkid = get_blkid(block_hash)
    for tx in block['tx']:
        txid = get_txid(tx['hash'])
        n = -1
        for vout in tx['vout']:
            n = n + 1
            oaddrs = list()
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'witness_unknown', 'multisig'):
                oaddrs.extend(vout['scriptPubKey']['addresses'])
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard', 'nulldata'):
                try:
                    oaddrs.extend([pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    oaddrs.extend([f'{tx["hash"]}{n}'])
            else:
                raise Exception(f'BUG!! {theight}\n{tx}\n{vout}')
            for addr in oaddrs:
                addrid = get_addrid(addr)
                oaddresses.append((txid, n, addrid))
    return oaddresses

In [ ]:
import time
import pickle
import itertools
import multiprocessing

if os.path.exists('taking_txinhash.pickle'):
    with open('taking_txinhash.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = dict()

term = 1000
start_height = get_meta('txin_height')
if start_height is not None:
    start_height = int(start_height)
if start_height is None or start_height < term:
    start_height = 0
else:
    start_height = start_height - term + 1
end_height = get_max_height()
print(f'Start from {start_height} to {end_height}')

pool_num = multiprocessing.cpu_count()//2

try:
    journal_mode('WAL')
    for sheight, eheight in zip(range(start_height, end_height, term), range(start_height+term, end_height+term, term)):
        stime = time.time()
        if eheight >= end_height:
            eheight = end_height+1
        begin_transactions()
        with multiprocessing.Pool(pool_num) as p:
            results = p.imap(get_addresses_in, range(sheight, eheight))
            for iaddresses in results:
                insert_txins(iaddresses)
        update_meta('txin_height', eheight-1)
        commit_transactions()
        etime = time.time()
        taking[sheight] = etime-stime
        print(f'TxIn Job done from {sheight} to {eheight-1} during {etime-stime}')
except KeyboardInterrupt:
    print(f'KeyboardInterrupt')
    commit_transactions()
finally:
    journal_mode('DELETE')
    with open('taking_txinhash.pickle', 'wb') as f:
        pickle.dump(taking, f)
    conn.close()
    iconn.close()